In [4]:
import sys
import os
from config import BASE_DIR
sys.path.insert(0, os.path.join(BASE_DIR, "model"))

In [5]:
from dotenv import load_dotenv
import torch

load_dotenv('.env')

True

In [6]:
# Load model as a PyFuncModel.
model = torch.load(os.path.join(BASE_DIR, "model.pth"), map_location=torch.device('cpu'))

C:\Users\user\AppData\Local\Temp\ipykernel_3256\198102879.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model = torch.load(os.path.join(BASE_DIR, "model.pth"), map_loc

In [7]:
model.eval()

GKT(
  (graph_model): MultiHeadAttention(
    (w_qs): Linear(in_features=32, out_features=64, bias=False)
    (w_ks): Linear(in_features=32, out_features=64, bias=False)
    (attention): ScaledDotProductAttention()
  )
  (emb_x): Embedding(1092, 32)
  (emb_c): Embedding(92, 32, padding_idx=91)
  (f_self): MLP(
    (fc1): Linear(in_features=64, out_features=32, bias=True)
    (fc2): Linear(in_features=32, out_features=32, bias=True)
    (norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (f_neighbor_list): ModuleList(
    (0-1): 2 x MLP(
      (fc1): Linear(in_features=128, out_features=32, bias=True)
      (fc2): Linear(in_features=32, out_features=32, bias=True)
      (norm): BatchNorm1d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (erase_add_gate): EraseAddGate(
    (erase): Linear(in_features=32, out_features=32, bias=True)
    (add): Linear(in_features=32, out_features=32, bias=True)
  )
  (gru): GRUCell(3

In [8]:
import pandas as pd


# 문자열 데이터를 데이터프레임으로 변환
data_frame = pd.read_parquet('filtered_combined_user_data.parquet')

# 데이터프레임 출력
data_frame

,UserID,mCode,QuizCode,Correct,CreDate,f_mchapter_id
0,a93858f1-ee72-4961-bc81-99cc56381106,T1ME11U01001,30140114,X,2024-01-01 00:04:00,14201779.0
1,a93858f1-ee72-4961-bc81-99cc56381106,T1ME11U01001,30140113,O,2024-01-01 00:04:00,14201779.0
2,a93858f1-ee72-4961-bc81-99cc56381106,T1ME11U01001,30140115,X,2024-01-01 00:04:00,14201779.0
3,a93858f1-ee72-4961-bc81-99cc56381106,T1ME11U01001,30140110,O,2024-01-01 00:04:00,14201779.0
4,a93858f1-ee72-4961-bc81-99cc56381106,T1ME11U01001,30140044,X,2024-01-01 00:04:00,14201779.0
...,...,...,...,...,...,...
12885738,d7a24bd1-a1c5-4e36-a8de-e36a2ebafea6,T1ME22U02006,30159257,X,2024-09-30 23:58:00.000,14201893.0
12885739,d7a24bd1-a1c5-4e36-a8de-e36a2ebafea6,T1ME22U02006,30159256,X,2024-09-30 23:58:00.000,14201893.0
12885740,d7a24bd1-a1c5-4e36-a8de-e36a2ebafea6,T1ME22U02006,30159255,O,2024-09-30 23:58:00.000,14201893.0
12885741,d7a24bd1-a1c5-4e36-a8de-e36a2ebafea6,T1ME22U02006,30159254,X,2024-09-30 23:58:00.000,14201893.0


In [9]:
data_frame['UserID'].value_counts()

UserID
75827c22-6895-4c9d-83e4-9b7cb21116e9    1867
82e9fc77-17ce-44f9-893b-e0442b742ab3    1750
1d4850ac-1977-4309-a6e8-9535b743f407    1550
fa7ab926-db6e-4e9b-accc-b56f341db193    1540
a3f780b0-d1ef-47ab-848c-c7f72949589c    1506
                                        ... 
c1bfc83f-8b7d-46f0-8016-a9a7b5372d11       1
8be37e3b-5222-4250-a622-f2043a65af45       1
8cc0857d-d5fa-4766-a2f0-4b4b0be89341       1
617b65e6-2849-4986-90a3-96bb6309d3e5       1
9d7f62db-dd80-473e-ab4c-625cbf2057b7       1
Name: count, Length: 56565, dtype: int64

In [119]:
# 데이터프레임에서 유저 데이터 로드
df = data_frame
user_id = 'a3f780b0-d1ef-47ab-848c-c7f72949589c'
user_data = df[df['UserID'] == user_id]

# Step 0 - 정렬: 가장 오래된 기록부터 정렬
user_data.sort_values(by=["UserID", "CreDate"], inplace=True)  # "CreDate" 컬럼을 기준으로 정렬

# Step 2 - Enumerate skill id
user_data['skill'], _ = pd.factorize(user_data['f_mchapter_id'], sort=True)  # we can also use problem_id to represent exercises

# correct 생성 (O -> 1, X -> 0)
user_data['Correct'] = user_data['Correct'].map({'O': 1, 'X': 0})

# Step 3 - Cross skill id with answer to form a synthetic feature
# use_binary: (0,1); !use_binary: (1,2,3,4,5,6,7,8,9,10,11,12). Either way, the correct result index is guaranteed to be 1
user_data['skill_with_answer'] = user_data['skill'] * 2 + user_data['Correct']

print(user_data.tail(10))


# 유저 풀이 시퀀스 및 다음 문제 정의
features = user_data['skill_with_answer'].tolist()
questions = user_data['skill'].tolist()

                                        UserID         mCode  QuizCode  \
12785272  a3f780b0-d1ef-47ab-848c-c7f72949589c  T1ME22U02064  30159326   
12785320  a3f780b0-d1ef-47ab-848c-c7f72949589c  T1ME22U02063  30159353   
12785323  a3f780b0-d1ef-47ab-848c-c7f72949589c  T1ME22U02063  30159354   
12785325  a3f780b0-d1ef-47ab-848c-c7f72949589c  T1ME22U02063  30159266   
12785332  a3f780b0-d1ef-47ab-848c-c7f72949589c  T1ME22U02063  30159265   
12874284  a3f780b0-d1ef-47ab-848c-c7f72949589c  T1ME22U02012  30159316   
12874304  a3f780b0-d1ef-47ab-848c-c7f72949589c  T1ME22U02012  30159317   
12874354  a3f780b0-d1ef-47ab-848c-c7f72949589c  T1ME22U02012  30159314   
12874357  a3f780b0-d1ef-47ab-848c-c7f72949589c  T1ME22U02012  30159313   
12874359  a3f780b0-d1ef-47ab-848c-c7f72949589c  T1ME22U02012  30159315   

          Correct                  CreDate  f_mchapter_id  skill  \
12785272        1  2024-09-28 12:32:00.000     14201894.0     45   
12785320        1  2024-09-28 12:34:00.000     14

C:\Users\user\AppData\Local\Temp\ipykernel_3256\2803419251.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data.sort_values(by=["UserID", "CreDate"], inplace=True)  # "CreDate" 컬럼을 기준으로 정렬
C:\Users\user\AppData\Local\Temp\ipykernel_3256\2803419251.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  user_data['skill'], _ = pd.factorize(user_data['f_mchapter_id'], sort=True)  # we can also use problem_id to represent exercises
C:\Users\user\AppData\Local\Temp\ipykernel_3256\2803419251.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy o

In [122]:
next_skills = list(user_data['skill'])
next_answers = list(user_data['Correct'])

In [123]:
next_skills

[23,
 23,
 23,
 24,
 24,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 26,
 26,
 26,
 26,
 26,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 27,
 28,
 28,
 28,
 28,
 28,
 28,
 28,
 30,
 34,
 34,
 34,
 36,
 36,
 36,
 36,
 36,
 36,
 37,
 37,
 38,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 23,
 23,
 23,
 23,
 23,
 24,
 24,
 24,
 24,
 24,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 23,
 22,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 22,
 21,
 21,
 21,
 21,
 22,
 22,
 22,
 23,
 23,
 22,
 22,
 22,
 22,
 22,
 22,
 22,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 21,
 22,
 22,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 25,
 26,
 26,
 26,
 26,
 26,
 22,
 22,
 22,
 22,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 23,
 21,
 24,
 23,
 22,
 21,
 23,
 22,
 24,
 22,
 22,
 24,
 22,
 22,
 23,
 24,
 22,
 24,
 22,
 22,
 22,
 24,
 22,
 22,
 24,
 24,
 22,
 23,
 22,
 22,
 22,
 27,
 27,
 27,
 27,
 27,
 25,
 26,
 26,


In [124]:
next_mchapter_ids = list(user_data['f_mchapter_id'])

In [125]:
next_answers

[1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,


In [126]:
# 데이터 텐서로 변환 및 디바이스 이동
features_tensor = torch.tensor(features, dtype=torch.long).unsqueeze(0).to('cpu')
questions_tensor = torch.tensor(questions, dtype=torch.long).unsqueeze(0).to('cpu')

In [127]:
with torch.no_grad():
    pred_res, _, _, _ = model(features_tensor, questions_tensor)  # 입력값과 동일한 디바이스에서 수행
    next_preds = pred_res.squeeze(0)[:]  # 다음 문제에 해당하는 예측값

In [128]:
next_preds

tensor([0.7000, 0.7703, 0.5825,  ..., 0.9126, 0.9320, 0.9425])

In [129]:
import numpy as np

# 기존 threshold 대신 여러 threshold를 계산하기 위해 범위 설정
thresholds = np.linspace(0, 1, 101)  # 0.00에서 1.00까지 0.01 간격
accuracies = []

# Threshold 별 정확도 계산
for threshold in thresholds:
    correct_predictions = 0
    total_predictions = len(next_preds)
    
    for i, pred in enumerate(next_preds.tolist()):
        pred_result = 1 if pred >= threshold else 0
        if pred_result == next_answers[i]:
            correct_predictions += 1
    
    # 정확도 저장
    accuracy = correct_predictions / total_predictions
    accuracies.append(accuracy)

# 최적 threshold 선택
best_threshold = thresholds[np.argmax(accuracies)]
# best_threshold = 1
best_accuracy = max(accuracies)

print(f"최적 Threshold: {best_threshold:.2f}, 정확도: {best_accuracy:.2f}")

# 최적 threshold를 기반으로 분석 출력
print("\n다음에 풀 10문제의 예측값 및 정오표 (최적 Threshold 기준):")
for i, pred in enumerate(next_preds.tolist()):
    # 최적 threshold에 따라 예측 결과 결정
    pred_result = 1 if pred >= best_threshold else 0
    
    # 실제 결과와 예측 결과 비교
    if pred_result == next_answers[i]:
        if pred_result == 1:
            analysis = "개념 확립 (정답 확신)"
        else:
            analysis = "개념 확립 (오답 확신)"
    else:
        if pred_result == 1:
            analysis = "실수 (과신)"
        else:
            analysis = "찍음 (운 좋게 맞춤)"
    
    # 결과 출력
    print(f"문제 {next_skills[i]}: 정답 확률 = {pred:.4f} ({pred_result}) 실제 = {next_answers[i]} -> 분석: {analysis}")


최적 Threshold: 0.51, 정확도: 0.88

다음에 풀 10문제의 예측값 및 정오표 (최적 Threshold 기준):
문제 23: 정답 확률 = 0.7000 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 23: 정답 확률 = 0.7703 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 23: 정답 확률 = 0.5825 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 24: 정답 확률 = 0.6046 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 24: 정답 확률 = 0.7635 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 25: 정답 확률 = 0.8093 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 25: 정답 확률 = 0.8355 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 25: 정답 확률 = 0.8466 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 25: 정답 확률 = 0.8417 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 25: 정답 확률 = 0.8237 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 25: 정답 확률 = 0.8042 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 25: 정답 확률 = 0.7934 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 25: 정답 확률 = 0.8958 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 26: 정답 확률 = 0.9060 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 26: 정답 확률 = 0.9122 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 26: 정답 확률 = 0.9166 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 26: 정답 확률 = 0.9201 (1) 실제 = 1 -> 분석: 개념 확립 (정답 확신)
문제 26: 정답 

In [20]:
# threshold = 0.80

# print("다음에 풀 10문제의 예측값 및 정오표:")
# for i, pred in enumerate(next_preds.tolist()):
#     # 예측된 결과 결정
#     pred_result = 1 if pred >= threshold else 0
#     # 실제 결과와 예측 결과 비교
#     if pred_result == next_answers[i]:
#         if pred_result == 1:
#             analysis = "개념 확립 (정답 확신)"
#         else:
#             analysis = "개념 확립 (오답 확신)"
#     else:
#         if pred_result == 1:
#             analysis = "실수 (과신)"
#         else:
#             analysis = "찍음 (운 좋게 맞춤)"
    
#     # 결과 출력
#     print(f"문제 {next_skills[i]}: 정답 확률 = {pred:.4f} ({pred_result}) 실제 = {next_answers[i]} -> 분석: {analysis}")